# Step 4 NE204 Lab 1
## Height Finding Method Optimization Routine
__Dates: 08/2022 to 10/2022.__

__Group: Megan Schiferl, Chris Lamb, Curtis Berger, Jisu Park__

__Contents:__
This notebook takes in raw pulses and returns 3 csv files containing trapezoid height. The first file finds the heights using the maximum value method, the second uses a trapezoid fitting method, and the third uses a gradient method. This notebook is also used to save the heights of trapezoids using only the maximum value method for multiple sources (simply change the data file and don't run the last two cells)

     Section 1: Imports and Directory
     Section 2: Functions
     Section 3: Calculating Trapezoids
     Section 4: Height Finding Methods
     
__Notes on Running:__
As usual, you'll need to check your directory and the imported data name, but otherwise you should be able to run all. This one takes about 10 mins to run, but I also uploaded the csv files that I got from running this to github, so you don't have to take the time to run it. 

# Section 1: Imports and Directory

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import time
import pandas as pd
from scipy import signal
from scipy.signal import savgol_filter
from scipy.optimize import curve_fit
import os
from tqdm import notebook
import matplotlib.colors as mcolors
from decimal import Decimal

In [6]:
# change directory
os.chdir(r'C:/Users/megas/Documents/Cal/NEFall2022/Detectors204/lab1/')

#check current working directory
retval = os.getcwd()
print ("Current working directory %s" %retval)

Current working directory C:\Users\megas\Documents\Cal\NEFall2022\Detectors204\lab1


In [7]:
#Input the data file and find the keys
path = r'C:\Users\megas\Documents\Cal\NEFall2022\Detectors204\lab1\FinalData\Eu152-24in-5-001.h5'
    
f = h5py.File(path, 'r')

length = input("How long did you collect this data:")
isotope = input("What isotope did you measure?:")

How long did you collect this data:5mins
What isotope did you measure?:Eu152


In [8]:
#Save the data into an array for analysis
pulses = np.array(f['raw_data'])

f.close()
#Remove the last second of pulses due to DAC mistake (approximate)
pulses = pulses[:-140]

print("The number of pulses in this file is:", len(pulses))
print("The length of each pulse is:", len(pulses[0]))

The number of pulses in this file is: 33833
The length of each pulse is: 62500


In [9]:
#Split the large files up into different arrays
pulses1 = []
pulses2 = []
pulses3 = []
pulses4 = []
pulses5 = []
pulses6 = []
pulses7 = []
for i in notebook.tqdm(range(len(pulses))):
    if i < 5000:
        pulses1.append(pulses[i])
    elif i < 10000:
        pulses2.append(pulses[i])
    elif i < 15000:
        pulses3.append(pulses[i])
    elif i < 20000:
        pulses4.append(pulses[i])
    elif i < 25000:
        pulses5.append(pulses[i])
    elif i < 30000:
        pulses6.append(pulses[i])
    else:
        pulses7.append(pulses[i])

pulse_names = [pulses1, pulses2, pulses3, pulses4, pulses5, pulses6, pulses7]
        
print(len(pulses1), len(pulses2), len(pulses3), len(pulses4), len(pulses5), len(pulses6), len(pulses7))

  0%|          | 0/33833 [00:00<?, ?it/s]

5000 5000 5000 5000 5000 5000 3833


# Section 2: Functions

In [10]:
#defining an exponential function
def exp_func(x, a, b, c):
    #returns a times e^(-b times a) + c
    return a * np.exp(-b * x) + c
    
#Finding the start of rise index using V4 from the notes notebook
def determine_rise(signal, sigma=8, window=20, offset=15):
    noise_samp = signal[:500]
    mean, std = np.mean(noise_samp), np.std(noise_samp)

    grad = np.gradient(np.where(signal > mean+sigma*std, signal, 0))

    grad_pos, grad_neg = np.argwhere(grad>2), np.argwhere(grad<-2)

    rise_start = 0
    for gp in grad_pos:
        close = False
        for gn in grad_neg:
            if (gp-gn < window and gp-gn >0):
                close = True
        if not close:
            rise_start = gp
            break

    return int(rise_start-offset)

#Define the trapezoidal filter and signal output function
def delay_signal(signal, delay, p=500):
    np.random.seed(9)
    noise_samp = signal[:p]
    mean, std = np.mean(noise_samp), np.std(noise_samp)
    noise = np.random.normal(mean, 0.8*std, delay)
    return np.hstack([noise, signal])

def dkl(signal, i, k, l, w=0):
    if w == 0:
        w = int(2.5*k+l)
    vj = signal[i+w:i+w+w] 
    vjk = signal[i+w-k:i+w-k+w]
    vjl = signal[i+w-l:i+w-l+w]
    vjkl = signal[i+w-k-l:i+w-k-l+w]
    dkl_s = vj - vjk - vjl + vjkl
    return dkl_s

def sfunc(signal, start_rise, tau, peaking_time, gap_time, w=0):
    if w == 0:
        w = int(2.5*peaking_time+gap_time)
    ss = []
    dkl_s = dkl(signal, start_rise, peaking_time, peaking_time+gap_time, w)
    for j in range(w):
        if j == 0:
            ss.append(0)
        else:
            ss.append(ss[j-1]*(1+1/tau)+ dkl_s[j])
    return np.array(ss)

def fit_trap(x, topleft, topright, top, ps, pe, e):
    #Trapezoid signal shape
    tr = np.zeros(len(x))
    tr[:int(ps)] = 0
    tr[int(ps):int(topleft)] = ((top-0)/(topleft-ps))*(x[int(ps):int(topleft)]-ps)+0
    tr[int(topleft):int(topright)] = top
    tr[int(topright):int(pe)] = ((e-top)/(pe-topright))*(x[int(topright):int(pe)]-pe)+e
    tr[int(pe):] = e
    return tr

# Section 3: Calculating Trapezoids

In [11]:
for i in notebook.tqdm(range(len(pulse_names)), desc = "Iteration Over Chunks of Pulses"):
    pulsesX = pulse_names[i]
    k = 950  #previously optimized peaking time 
    m = 1000  #previously optimized gap time
    l = k+m
    
    #remove saturated signals
    sat = np.amax(np.amax(pulsesX))

    nonsat_pulses = []
    for a in notebook.tqdm(range(len(pulsesX)), desc="Removing Saturated Pulses", leave = False):
        max_y = np.amax(pulsesX[a])
        if (max_y != sat):
            nonsat_pulses.append(pulsesX[a])

    nonsat_pulses = np.array(nonsat_pulses)   

    #Background subtraction for the raw data
    n = len(nonsat_pulses)
    pulses_sub = []
    for c in notebook.tqdm(range(n), desc = "Noise Floor Subtraction", leave = False):
        bkg = np.mean(nonsat_pulses[c][0:500])
        pulses_sub.append(nonsat_pulses[c] - bkg)

    pulses_sub = np.array(pulses_sub)

    #filtering the signals
    fs = savgol_filter(pulses_sub, 53, 2)

    #find the index of the max value for each fitted signal
    imax = []
    for q in notebook.tqdm(range(len(fs)), desc = "Finding index of max", leave = False):
        if np.argmax(fs[q][0:2000]) != 1999:
            imax.append(np.argmax(fs[q][0:2000])) # only looking at the first 2000 to avoid late pileup signals

    #fit an exponential curve to the decay of each signal to find tau
    if i == 0:
        tau = []
        for d in notebook.tqdm(range(100), desc = "Finding Tau", leave = False):
            #setting the ranges for exp fitting and x axis parameters
            endfit = 20000
            s = imax[d] #index of max for pulse d
            x = np.arange(0, endfit) #array of values from max index upward for some number of indices, len(x)=endfit

            #using exp_func over the values in x
            #fits an exp to the ith pulse from index of max to endfit+index of max
            try:
                popt, pcov = curve_fit(exp_func, x, fs[d][s:endfit+s])
                tau.append(1/popt[1])
            except:
                tau.append(0)

        tau = list(filter(lambda a: a != 0, tau))
        tau = np.array(tau)
        Tau = int(np.mean(tau))

    #Place all i in an array
    startrise = []
    for q2 in notebook.tqdm(range(len(fs)), desc = "Finding the start rise index", leave = False):
        itemp = int(determine_rise(fs[q2]))
        startrise.append(itemp)
    startrise = np.array(startrise)

    #Delay the signals 
    dfs = []
    d=int(2.5*k+m)
    for r in notebook.tqdm(range(len(fs)), desc = "Delaying Signals", leave = False):
        tempfs = delay_signal(fs[r],delay=d)
        dfs.append(tempfs)
    dfs = np.array(dfs)

    #Calculate the trapezoids
    traps = []
    for t in notebook.tqdm(range(len(dfs)), desc = "Calculating Trapezoids", leave = False):
        trap = sfunc(dfs[t], startrise[t], Tau, k, m)
        traps.append(trap)
        
traps = np.array(traps)

Iteration Over Chunks of Pulses:   0%|          | 0/7 [00:00<?, ?it/s]

Removing Saturated Pulses:   0%|          | 0/5000 [00:00<?, ?it/s]

Noise Floor Subtraction:   0%|          | 0/4992 [00:00<?, ?it/s]

Finding index of max:   0%|          | 0/4992 [00:00<?, ?it/s]

Finding Tau:   0%|          | 0/100 [00:00<?, ?it/s]

...

Noise Floor Subtraction:   0%|          | 0/3826 [00:00<?, ?it/s]

Finding index of max:   0%|          | 0/3826 [00:00<?, ?it/s]

Finding the start rise index:   0%|          | 0/3826 [00:00<?, ?it/s]

Delaying Signals:   0%|          | 0/3826 [00:00<?, ?it/s]

Calculating Trapezoids:   0%|          | 0/3826 [00:00<?, ?it/s]

# Section 4: Height Finding Methods

In [14]:
############################## Max Height Version Only ####################################################
height = []
for u in notebook.tqdm(range(len(traps)), desc = "Finding Heights", leave = False):
    height.append(np.amax(traps[u]))

height = list(filter(lambda a: a >= 0, height))    
allheights = allheights + height
        
allheights = np.array(allheights)
################################ End of Max Height ########################################################


############################ SAVING HEIGHTS ###############################################################
# change directory
os.chdir(r'C:\Users\megas\Documents\Cal\NEFall2022\Detectors204\lab1\TrapHeightsTEST')

#Let's just save these heights to a file
name = "{}_{}_k{}m{}_MaxMethod.csv".format(isotope,length,k,m)
np.savetxt(name, allheights, delimiter=",")

Finding Heights:   0%|          | 0/3826 [00:00<?, ?it/s]

In [16]:
####################### Trapezoid Fit Version Only ###################################################
h1 = []
for v1 in notebook.tqdm(range(len(traps)), leave = False):
    try:
        popt, pcov = curve_fit(fit_trap, np.arange(len(traps[v1])), traps[v1], p0=[400, 1200, 100000, 100, 1400, 1500])
        h1.append(popt)
    except:
        h1.append(np.zeros(6))

#Trapezoid Fit method cont.
h2 = []
#Remove the zero arrays from h1
for a in range(len(h1)):
    result = h1[a].all(0)
    if result == True:
        h2.append(h1[a])
h2 = np.array(h2)

#find the mean of the top of traps
height2 = []
for b in notebook.tqdm(range(len(h2)), leave = False):
    i1 = int(h2[b][3])
    i2 = int(h2[b][4])
    temph = np.mean(traps[b][i1:i2+1])
    height2.append(temph)
height2 = list(filter(lambda a: a >= 0, height2))
allheights2 = []
allheights2 = allheights2 + height2

allheights2 = np.array(allheights2)
############################ End of Trap Fit Version #####################################################

############################ SAVING HEIGHTS ###############################################################
# change directory
os.chdir(r'C:\Users\megas\Documents\Cal\NEFall2022\Detectors204\lab1\TrapHeightsTEST')

#Let's just save these heights to a file
name = "{}_{}_k{}m{}_TrapFitMethod.csv".format(isotope,length,k,m)
np.savetxt(name, allheights, delimiter=",")

  0%|          | 0/3826 [00:00<?, ?it/s]

  0%|          | 0/3826 [00:00<?, ?it/s]

In [18]:
################################ Start Gradient Method ###############################################
#Using the gradient of the trapezoid to find the flat top (instead of fitting a trapezoid to it)
bound1 = []
bound2 = []
tg = []
tgg = []
for j3 in notebook.tqdm(range(len(traps)), desc = "Taking Derivatives", leave = False):
    tgtemp = np.gradient(traps[j3])
    tggtemp = np.gradient(tgtemp)
    tg.append(tgtemp)
    tgg.append(tggtemp)
tg = np.array(tg)
tgg = np.array(tgg)
ftgg = savgol_filter(tgg, 7, 0)

for i2 in notebook.tqdm(range(len(tgg)), desc = "Finding Bounds of Trapezoid", leave = False):        
    sortedtgg = np.argsort(ftgg[i2])
    minima = []
    maxima = []
    for j2 in range(500):
        diff = np.abs(sortedtgg[j2]-sortedtgg[j2+1])
        if diff > 400:
            minima.append(sortedtgg[j2])
    minima = np.array(minima)

    #minima[0] and minima[1] are the trapezoid top boundaries
    b1 = min(minima[0], minima[1])
    b2 = max(minima[0], minima[1])
    bound1.append(b1)
    bound2.append(b2)

bound1 = np.array(bound1)
bound2 = np.array(bound2)

allzeros = []
for k2 in notebook.tqdm(range(len(ftgg)), desc = "Finding Zero Crossings", leave = False):  
    zeros = []
    for m2 in range(bound2[k2]-bound1[k2]):
        n2 = m2 + bound1[k2]
        if ftgg[k2][n2]*ftgg[k2][n2+1] <= 0:
            zeros.append(n2)
    allzeros.append(zeros)

allzeros = np.array(allzeros)

start = []
end = []
height3 = []
duds = []
for w in notebook.tqdm(range(len(allzeros)), desc = "Finding the Height", leave = False):
    try:
        st = allzeros[w][0]
        ed = allzeros[w][-1]
        start.append(st)
        end.append(ed)
    except:
        start.append(bound1[w])
        end.append(bound2[w])

    #find the mean height
    trapheight = np.mean(traps[w][st:ed])
    height3.append(trapheight)

start = np.array(start)
end = np.array(end)
allheights3 = []
allheights3 = allheights3 + height3

allheights3 = np.array(allheights3)
################################# End Gradient Method ####################################################

############################ SAVING HEIGHTS ###############################################################
# change directory
os.chdir(r'C:\Users\megas\Documents\Cal\NEFall2022\Detectors204\lab1\TrapHeightsTEST')

#Let's just save these heights to a file
name = "{}_{}_k{}m{}_GradientMethod.csv".format(isotope,length,k,m)
np.savetxt(name, allheights3, delimiter=",")

Taking Derivatives:   0%|          | 0/3826 [00:00<?, ?it/s]

Finding Bounds of Trapezoid:   0%|          | 0/3826 [00:00<?, ?it/s]

Finding Zero Crossings:   0%|          | 0/3826 [00:00<?, ?it/s]

C:\Users\megas\anaconda3\envs\NE204V1\lib\site-packages\ipykernel_launcher.py:44: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Finding the Height:   0%|          | 0/3826 [00:00<?, ?it/s]